In [ ]:
%%time
import datetime

start = datetime.datetime.now()
print(start)
import os as os

os.chdir("C:/Users/prana/Desktop/directory/WildAlertModels/")
# os.chdir('/home/falco/Desktop/directory/Syndrome_classifier_trainer')
data_path = "C:/Users/prana/Desktop/directory/WildAlertModels/data/"
# data_path = '/home/falco/Desktop/directory/Syndrome_classifier_trainer/data'
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline
import datetime
import os
import pickle
from math import ceil

import app
import config
import nltk
import numpy as np
import pandas as pd
import preprocess
import simplejson as json
from huggingface_hub import login
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from textblob import TextBlob
from tqdm import tqdm

# from WRMDpy import Syndrome as Syndrome

# nltk.download('stopwords')


# import tensorflow as tf
# from tensorflow.keras.models import Sequential,Model
# from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
# from tensorflow.keras.layers import Input,Bidirectional,GlobalAveragePooling1D,GlobalMaxPooling1D,concatenate,LeakyReLU
# from tensorflow.keras import regularizers
# from tensorflow.keras import backend as K
# from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
# import tensorflow as tf
# print("tensorflow version ", tf.__version__)

# import keras
# print("keras version ",keras.__version__)
# import keras.callbacks
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras_preprocessing.sequence import pad_sequences

login(token=None)

plt.style.use("fivethirtyeight")
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.family"] = "serif"

In [ ]:
import requests

path1 = "https://www.wrmd.org/api/v2/ml/training-dataset/clinical-classifications"
# path1 = 'API/link/TrainingData'
raw_data = json.loads(requests.get(path1).text)
raw_data = pd.DataFrame(raw_data)
cond = raw_data.terms.apply(pd.Series)
cond.columns = ["c1", "c2", "c3", "c4", "c5"]
raw_data = pd.concat([raw_data, cond], axis=1)
raw_data["condition_predict"] = raw_data.c1
raw_data.head()

In [ ]:
import re

df = raw_data
REPLACE_BY_SPACE_RE = re.compile("[/(){}\[\]\|@,;]")
BAD_SYMBOLS_RE = re.compile("[^0-9a-z #+_]")
STOPWORDS = set(stopwords.words("english"))


def clean_text(text):
    """
    text: a string

    return: modified initial string
    """
    text = text.lower()  # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(
        " ", text
    )  # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub(
        "", text
    )  # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing.
    text = text.replace("x", "")
    #    text = re.sub(r'\W+', '', text)
    text = " ".join(
        word for word in text.split() if word not in STOPWORDS
    )  # remove stopwors from text
    return text


df["text_clean"] = df["text"].apply(clean_text)
df["text_clean"] = df["text_clean"].str.replace("\d+", "")

In [ ]:
df["text_clean"]

### creating dictionary similar to imdb dataset
* Only one label : `condition_predict`
* text column as predictor: `text`

In [ ]:
raw_data_dict = df[["text_clean", "condition_predict"]].to_dict()

In [ ]:
raw_data_dict["text_clean"][0]

### Preprocess

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def tokenize_text(df, text_column):
    return df["text_clean"].apply(lambda x: tokenizer.encode(x, truncation=True))

In [ ]:
tokenized_wrmd = tokenize_text(raw_data, "text")

Now create a batch of examples using `DataCollatorWithPadding`. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Evaluate

Including a metric during training is often helpful for evaluating your model’s performance. You can quickly load a evaluation method with the 🤗 Evaluate library. For this task, load the accuracy metric (see the 🤗 Evaluate quick tour to learn more about how to load and compute a metric):

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

Then create a function that passes your predictions and labels to compute to calculate the accuracy:

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

### Label dictionaries

In [ ]:
conditions = raw_data.condition_predict.unique().tolist()
values = list(np.arange(len(conditions)))
values = [int(x) for x in values]
label2id = dict(zip(conditions, values))
print(label2id)

In [ ]:
id2label = {v: k for k, v in label2id.items()}
print(id2label)

In [ ]:
len(conditions)

## Training Pytorch model

In [ ]:
%%time
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(conditions),
    id2label=id2label,
    label2id=label2id,
)

Define your training hyperparameters in TrainingArguments.  
The only required parameter is output_dir which specifies where to save your model.   
You’ll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model).   
At the end of each epoch, the Trainer will evaluate the accuracy and save the training checkpoint.  
Pass the training arguments to Trainer along with the model, dataset, tokenizer, data collator, and compute_metrics function.  
Call `train()` to finetune your model.  

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wrmd,
    eval_dataset=tokenized_wrmd,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
tokenized_wrmd